In [1]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

api_key = os.environ.get("cle_gemini_api")
if not api_key:
    raise ValueError("La clé API n'est pas définie dans le fichier .env. Veuillez configurer la variable 'cle_api'.")

# Configurer le module avec la clé API
genai.configure(api_key=api_key)

# Exemple d'utilisation de l'API
for m in genai.list_models():
    print(m)

/home/maximeadmin/anaconda3/envs/mon_env_ia/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model(name='models/chat-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 Chat (Legacy)',
      description='A legacy text-only model optimized for chat conversations',
      input_token_limit=4096,
      output_token_limit=1024,
      supported_generation_methods=['generateMessage', 'countMessageTokens'],
      temperature=0.25,
      max_temperature=None,
      top_p=0.95,
      top_k=40)
Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 (Legacy)',
      description='A legacy model that understands text and generates text as an output',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      max_temperature=None,
      top_p=0.95,
      top_k=40)
Model(name='models/embedding-gecko-001',
      base_model_id='',
      version='001',
      display_name='Embedding Gecko

In [9]:
# Importation des bibliothèques nécessaires
import os
import re
import json
import time
from pathlib import Path
from dotenv import load_dotenv
import google.generativeai as genai

# Charger les variables d'environnement depuis le fichier .env
load_dotenv()
api_key = os.environ.get("cle_gemini_api")
if not api_key:
    raise ValueError("La clé API n'est pas définie dans le fichier .env. Veuillez configurer la variable 'cle_gemini_api'.")

# Configurer l'API Gemini
genai.configure(api_key=api_key)

def extract_uuid(content):
    """Extraire l'UUID du contenu LaTeX."""
    uuid_match = re.search(r'\\uuid\{([^}]+)\}', content)
    return uuid_match.group(1) if uuid_match else None

def create_prompt(latex_content):
    """Créer le prompt pour l'API Gemini."""
    return f"""Voici un exercice de mathématiques au format LaTeX. Merci de générer les métadonnées suivantes:
1. Compétences mathématiques requises pour résoudre l'exercice, listées par ordre d'importance (5 maximum)
2. Niveau de difficulté (facile, intermédiaire, avancé)
3. Mots-clés pertinents pour la recherche (8 maximum)
4. Concepts théoriques fondamentaux mobilisés (3-5)
5. Prérequis nécessaires pour aborder l'exercice
6. Type d'exercice (Exercice d'application directe, Problème à étapes, problème ouvert, Démonstration, Modélisation, Investigation)
7. Temps estimé pour la résolution

Réponds uniquement en format JSON structuré avec ces 7 clés exactes, dans cet ordre:
- "competences"
- "niveau_difficulte"
- "mots_cles"
- "concepts_fondamentaux"
- "prerequis"
- "type_exercice"
- "temps_estime"

Voici l'exercice:
{latex_content}"""

def display_response_details(text_response):
    """Afficher les détails de la réponse pour le débogage."""
    print("\n=== RÉPONSE BRUTE DE L'API ===")
    print(text_response)
    print("==============================\n")
    
    # Essayer plusieurs méthodes d'extraction pour voir ce qui fonctionne
    print("Tentatives d'extraction JSON:")
    
    # Méthode 1: JSON direct
    try:
        json_direct = json.loads(text_response)
        print("✓ Méthode 1 (JSON direct): Succès")
        print(json.dumps(json_direct, indent=2, ensure_ascii=False))
    except:
        print("✗ Méthode 1 (JSON direct): Échec")
    
    # Méthode 2: Bloc JSON markdown
    json_match = re.search(r'```json\n([\s\S]*?)\n```', text_response)
    if json_match:
        try:
            json_block = json.loads(json_match.group(1))
            print("✓ Méthode 2 (Bloc JSON markdown): Succès")
            print(json.dumps(json_block, indent=2, ensure_ascii=False))
        except:
            print("✗ Méthode 2 (Bloc JSON markdown): Échec")
    else:
        print("✗ Méthode 2 (Bloc JSON markdown): Pas de bloc trouvé")
    
    # Méthode 3: Bloc code générique
    json_match = re.search(r'```\n([\s\S]*?)\n```', text_response)
    if json_match:
        try:
            json_block = json.loads(json_match.group(1))
            print("✓ Méthode 3 (Bloc code générique): Succès")
            print(json.dumps(json_block, indent=2, ensure_ascii=False))
        except:
            print("✗ Méthode 3 (Bloc code générique): Échec")
    else:
        print("✗ Méthode 3 (Bloc code générique): Pas de bloc trouvé")
    
    # Méthode 4: Expression régulière pour trouver tout ce qui ressemble à un JSON
    json_match = re.search(r'\{[\s\S]*\}', text_response)
    if json_match:
        try:
            json_regex = json.loads(json_match.group(0))
            print("✓ Méthode 4 (Regex JSON): Succès")
            print(json.dumps(json_regex, indent=2, ensure_ascii=False))
        except:
            print("✗ Méthode 4 (Regex JSON): Échec")
    else:
        print("✗ Méthode 4 (Regex JSON): Pas de JSON trouvé")

def extract_json_from_response(text_response, debug=False):
    """Extraire le JSON de la réponse de l'API."""
    if debug:
        display_response_details(text_response)
    
    # Tenter différentes méthodes d'extraction
    try:
        # Méthode 1: JSON direct
        return json.loads(text_response)
    except json.JSONDecodeError:
        # Méthode 2: Bloc JSON markdown
        json_match = re.search(r'```json\n([\s\S]*?)\n```', text_response)
        if json_match:
            try:
                return json.loads(json_match.group(1))
            except:
                pass
        
        # Méthode 3: Bloc code générique
        json_match = re.search(r'```\n([\s\S]*?)\n```', text_response)
        if json_match:
            try:
                return json.loads(json_match.group(1))
            except:
                pass
        
        # Méthode 4: Expression régulière pour trouver tout ce qui ressemble à un JSON
        json_match = re.search(r'\{[\s\S]*\}', text_response)
        if json_match:
            try:
                return json.loads(json_match.group(0))
            except:
                pass
        
        # Si toutes les méthodes échouent
        if debug:
            print("ERREUR: Impossible d'extraire un JSON valide de la réponse.")
        raise ValueError("Format de réponse non reconnu ou JSON invalide")

def analyze_latex_file(file_path, output_dir="output", model_name="gemini-2.0-flash", force_reprocess=False, debug=False):
    """Analyser un fichier LaTeX et générer les métadonnées via l'API Gemini."""
    try:
        # Lire le contenu du fichier
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Extraire l'UUID
        uuid = extract_uuid(content)
        if not uuid:
            print(f"UUID non trouvé dans le fichier {file_path}")
            return None
        
        # Vérifier si le fichier a déjà été traité
        os.makedirs(output_dir, exist_ok=True)
        output_file = os.path.join(output_dir, f"{uuid}.json")
        if os.path.exists(output_file) and not force_reprocess:
            print(f"Fichier {uuid} déjà traité, ignoré.")
            return {uuid: json.load(open(output_file, 'r', encoding='utf-8'))[uuid]}
        
        # Créer le prompt
        prompt = create_prompt(content)
        
        if debug:
            print("\n=== PROMPT ENVOYÉ À L'API ===")
            print(prompt)
            print("============================\n")
        
        # Appeler l'API Gemini
        try:
            model = genai.GenerativeModel(model_name)
            response = model.generate_content(prompt)
            text_response = response.text
        except Exception as e:
            print(f"ERREUR lors de l'appel à l'API Gemini: {e}")
            return None
        
        # Extraire le JSON de la réponse
        try:
            json_data = extract_json_from_response(text_response, debug)
            
            # Vérifier si les données sont vides ou nulles
            if not json_data:
                print("AVERTISSEMENT: Les données JSON extraites sont vides.")
                if debug:
                    print("JSON extrait:", json_data)
                return None
            
            # Créer l'objet de résultat final avec vérification de l'existence des clés
            metadata = {
                "competences": json_data.get("competences") or json_data.get("1") or [],
                "niveau_difficulte": json_data.get("niveau_difficulte") or json_data.get("2") or "",
                "mots_cles": json_data.get("mots_cles") or json_data.get("3") or [],
                "concepts_fondamentaux": json_data.get("concepts_fondamentaux") or json_data.get("4") or [],
                "prerequis": json_data.get("prerequis") or json_data.get("5") or [],
                "type_exercice": json_data.get("type_exercice") or json_data.get("6") or "",
                "temps_estime": json_data.get("temps_estime") or json_data.get("7") or ""
            }
            
            # Vérifier si les données sont effectivement vides après traitement
            empty_fields = [k for k, v in metadata.items() if not v]
            if empty_fields:
                print(f"AVERTISSEMENT: Les champs suivants sont vides: {', '.join(empty_fields)}")
                if debug:
                    print("Métadonnées extraites:", metadata)
            
            result = {uuid: metadata}
            
            # Écrire le résultat dans un fichier
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(result, f, ensure_ascii=False, indent=2)
            
            return result
        except Exception as e:
            print(f"Erreur lors de l'extraction des métadonnées pour {uuid}: {e}")
            if debug:
                print(f"Réponse brute: {text_response}")
            return None
    except Exception as e:
        print(f"Erreur lors de l'analyse du fichier {file_path}: {e}")
        return None

def process_latex_files(source_dir="src/latex/amscc", output_dir="output", model_name="gemini-2.0-flash", 
                       max_files=None, api_delay=1.0, force_reprocess=False, debug=False):
    """Traiter tous les fichiers LaTeX dans le répertoire source."""
    # Créer le répertoire de sortie s'il n'existe pas
    os.makedirs(output_dir, exist_ok=True)
    
    # Obtenir tous les fichiers .tex dans le répertoire source
    files = [str(f) for f in Path(source_dir).glob('**/*.tex')]
    
    # Limiter le nombre de fichiers si demandé
    if max_files:
        files = files[:max_files]
        print(f"Mode test: limité à {max_files} fichiers")
    
    print(f"Traitement de {len(files)} fichiers LaTeX...")
    
    # Traiter chaque fichier
    results = {}
    success_count = 0
    
    for i, file in enumerate(files):
        print(f"\n[{i+1}/{len(files)}] Analyse de {os.path.basename(file)}...")
        result = analyze_latex_file(file, output_dir, model_name, force_reprocess, debug)
        
        if result:
            results.update(result)
            success_count += 1
        
        # Pause entre les appels API (sauf pour le dernier fichier)
        if i < len(files) - 1:
            print(f"Pause de {api_delay} secondes...")
            time.sleep(api_delay)
    
    # Écrire tous les résultats dans un seul fichier
    with open(os.path.join(output_dir, 'amscc_metadata.json'), 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    print(f"Traitement terminé. {success_count}/{len(files)} fichiers analysés avec succès.")
    return results

# Fonction simplifiée pour tester rapidement sur un seul fichier avec débogage activé
def test_single_file(file_path):
    """Tester l'analyse sur un seul fichier avec débogage complet."""
    print(f"Test d'analyse sur le fichier: {file_path}")
    result = analyze_latex_file(file_path, debug=True)
    if result:
        print("\n=== RÉSULTAT FINAL ===")
        print(json.dumps(result, indent=2, ensure_ascii=False))
        print("=====================\n")
    return result

In [11]:
results = process_latex_files(
    source_dir="../src/latex/amscc",  # Chemin vers vos fichiers LaTeX
    output_dir="../metadata/amscc",           # Répertoire de sortie
    #max_files=2,                   # Limiter à 3 fichiers pour le test
    api_delay=1.5                  # Pause de 1.5 secondes entre les appels API
)

Traitement de 460 fichiers LaTeX...

[1/460] Analyse de YBwt.tex...
Pause de 1.5 secondes...

[2/460] Analyse de trhY.tex...
Pause de 1.5 secondes...

[3/460] Analyse de aWAS.tex...
Pause de 1.5 secondes...

[4/460] Analyse de DEZs.tex...
Pause de 1.5 secondes...

[5/460] Analyse de 5cbY.tex...
Pause de 1.5 secondes...

[6/460] Analyse de ouYw.tex...
Pause de 1.5 secondes...

[7/460] Analyse de 6NIK.tex...
Pause de 1.5 secondes...

[8/460] Analyse de PPhD.tex...
Pause de 1.5 secondes...

[9/460] Analyse de 0oHk.tex...
Pause de 1.5 secondes...

[10/460] Analyse de hWFx.tex...
Pause de 1.5 secondes...

[11/460] Analyse de Uijb.tex...
Pause de 1.5 secondes...

[12/460] Analyse de 7aqI.tex...
Pause de 1.5 secondes...

[13/460] Analyse de cEVS.tex...
Pause de 1.5 secondes...

[14/460] Analyse de tNNT.tex...
Pause de 1.5 secondes...

[15/460] Analyse de gsn9.tex...
Pause de 1.5 secondes...

[16/460] Analyse de yp7e.tex...
Pause de 1.5 secondes...

[17/460] Analyse de cLt6.tex...
Pause de 1.5

In [12]:
# Importation des bibliothèques nécessaires
import os
import re
import json
import time
from pathlib import Path
from dotenv import load_dotenv
import google.generativeai as genai

# Charger les variables d'environnement depuis le fichier .env
load_dotenv()
api_key = os.environ.get("cle_gemini_api")
if not api_key:
    raise ValueError("La clé API n'est pas définie dans le fichier .env. Veuillez configurer la variable 'cle_gemini_api'.")

# Configurer l'API Gemini
genai.configure(api_key=api_key)

def extract_uuid(content):
    """Extraire l'UUID du contenu LaTeX."""
    uuid_match = re.search(r'\\uuid\{([^}]+)\}', content)
    return uuid_match.group(1) if uuid_match else None

def create_prompt(latex_content):
    """Créer le prompt pour l'API Gemini avec une attention particulière aux compétences."""
    return f"""Voici un exercice de mathématiques au format LaTeX. Analyse-le et génère les métadonnées suivantes:

1. Compétences mathématiques requises pour résoudre l'exercice (5 maximum)
   - Exprime chaque compétence sous forme d'un verbe d'action suivi d'un complément précis
   - Utilise des verbes comme: calculer, démontrer, dériver, résoudre, appliquer, interpréter, modéliser
   - Fais correspondre la compétence à une action concrète que l'étudiant doit maîtriser
   - Liste les compétences par ordre d'importance (la plus cruciale en premier)
   - Exemple: "calculer une dérivée partielle" plutôt que "dérivée partielle"

2. Niveau de difficulté (facile, intermédiaire, avancé)

3. Mots-clés pertinents pour la recherche (8 maximum)

4. Concepts théoriques fondamentaux mobilisés (3-5)

5. Prérequis nécessaires pour aborder l'exercice

6. Type d'exercice (Exercice d'application directe, Problème à étapes, problème ouvert, Démonstration, Modélisation, Investigation)

7. Temps estimé pour la résolution

Réponds uniquement en format JSON structuré avec ces 7 clés exactes:
{{
  "competences": ["compétence 1", "compétence 2", ...],
  "niveau_difficulte": "niveau",
  "mots_cles": ["mot-clé 1", "mot-clé 2", ...],
  "concepts_fondamentaux": ["concept 1", "concept 2", ...],
  "prerequis": ["prérequis 1", "prérequis 2", ...],
  "type_exercice": "type",
  "temps_estime": "temps"
}}

Voici l'exercice:
{latex_content}"""

def extract_json_from_response(text_response, debug=False):
    """Extraire le JSON de la réponse de l'API."""
    if debug:
        print("\n=== RÉPONSE BRUTE DE L'API ===")
        print(text_response)
        print("==============================\n")
    
    # Tenter différentes méthodes d'extraction
    try:
        # Méthode 1: JSON direct
        return json.loads(text_response)
    except json.JSONDecodeError:
        # Méthode 2: Bloc JSON markdown
        json_match = re.search(r'```json\n([\s\S]*?)\n```', text_response)
        if json_match:
            try:
                return json.loads(json_match.group(1))
            except:
                pass
        
        # Méthode 3: Bloc code générique
        json_match = re.search(r'```\n([\s\S]*?)\n```', text_response)
        if json_match:
            try:
                return json.loads(json_match.group(1))
            except:
                pass
        
        # Méthode 4: Expression régulière pour trouver tout ce qui ressemble à un JSON
        json_match = re.search(r'\{[\s\S]*\}', text_response)
        if json_match:
            try:
                return json.loads(json_match.group(0))
            except:
                pass
        
        # Si toutes les méthodes échouent
        if debug:
            print("ERREUR: Impossible d'extraire un JSON valide de la réponse.")
        raise ValueError("Format de réponse non reconnu ou JSON invalide")

def analyze_latex_file(file_path, output_dir="output", model_name="gemini-2.0-flash", force_reprocess=False, debug=False):
    """Analyser un fichier LaTeX et générer les métadonnées via l'API Gemini."""
    try:
        # Lire le contenu du fichier
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Extraire l'UUID
        uuid = extract_uuid(content)
        if not uuid:
            print(f"UUID non trouvé dans le fichier {file_path}")
            return None
        
        # Vérifier si le fichier a déjà été traité
        os.makedirs(output_dir, exist_ok=True)
        output_file = os.path.join(output_dir, f"{uuid}.json")
        if os.path.exists(output_file) and not force_reprocess:
            print(f"Fichier {uuid} déjà traité, ignoré.")
            with open(output_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        
        # Créer le prompt
        prompt = create_prompt(content)
        
        if debug:
            print("\n=== PROMPT ENVOYÉ À L'API ===")
            print(prompt)
            print("============================\n")
        
        # Appeler l'API Gemini
        try:
            model = genai.GenerativeModel(model_name)
            response = model.generate_content(prompt)
            text_response = response.text
        except Exception as e:
            print(f"ERREUR lors de l'appel à l'API Gemini: {e}")
            return None
        
        # Extraire le JSON de la réponse
        try:
            json_data = extract_json_from_response(text_response, debug)
            
            # Vérifier si les données sont vides ou nulles
            if not json_data:
                print("AVERTISSEMENT: Les données JSON extraites sont vides.")
                if debug:
                    print("JSON extrait:", json_data)
                return None
            
            # Créer l'objet de résultat final avec vérification de l'existence des clés
            metadata = {
                "competences": json_data.get("competences") or [],
                "niveau_difficulte": json_data.get("niveau_difficulte") or "",
                "mots_cles": json_data.get("mots_cles") or [],
                "concepts_fondamentaux": json_data.get("concepts_fondamentaux") or [],
                "prerequis": json_data.get("prerequis") or [],
                "type_exercice": json_data.get("type_exercice") or "",
                "temps_estime": json_data.get("temps_estime") or ""
            }
            
            # Vérifier si les compétences sont au format attendu
            if metadata["competences"]:
                # Vérifier que chaque compétence commence par un verbe d'action
                verbs = ["calculer", "démontrer", "dériver", "résoudre", "appliquer", "interpréter", 
                         "modéliser", "analyser", "utiliser", "évaluer", "étudier", "déterminer", 
                         "construire", "vérifier", "justifier", "comprendre", "explorer", "transformer"]
                
                for i, comp in enumerate(metadata["competences"]):
                    # Extraire le premier mot (potentiellement un verbe)
                    first_word = comp.lower().split()[0]
                    
                    # Si ce n'est pas un verbe d'action, préfixer avec un verbe approprié
                    if first_word not in verbs and not any(comp.lower().startswith(v) for v in verbs):
                        print(f"AVERTISSEMENT: La compétence '{comp}' ne commence pas par un verbe d'action.")
            
            result = {uuid: metadata}
            
            # Écrire le résultat dans un fichier
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(result, f, ensure_ascii=False, indent=2)
            
            return result
        except Exception as e:
            print(f"Erreur lors de l'extraction des métadonnées pour {uuid}: {e}")
            if debug:
                print(f"Réponse brute: {text_response}")
            return None
    except Exception as e:
        print(f"Erreur lors de l'analyse du fichier {file_path}: {e}")
        return None

def process_latex_files(source_dir="src/latex/amscc", output_dir="output", model_name="gemini-2.0-flash", 
                       max_files=None, api_delay=1.0, force_reprocess=False, debug=False):
    """Traiter tous les fichiers LaTeX dans le répertoire source."""
    # Créer le répertoire de sortie s'il n'existe pas
    os.makedirs(output_dir, exist_ok=True)
    
    # Obtenir tous les fichiers .tex dans le répertoire source
    files = [str(f) for f in Path(source_dir).glob('**/*.tex')]
    
    # Limiter le nombre de fichiers si demandé
    if max_files:
        files = files[:max_files]
        print(f"Mode test: limité à {max_files} fichiers")
    
    print(f"Traitement de {len(files)} fichiers LaTeX...")
    
    # Traiter chaque fichier
    results = {}
    success_count = 0
    
    for i, file in enumerate(files):
        print(f"\n[{i+1}/{len(files)}] Analyse de {os.path.basename(file)}...")
        result = analyze_latex_file(file, output_dir, model_name, force_reprocess, debug)
        
        if result:
            results.update(result)
            success_count += 1
        
        # Pause entre les appels API (sauf pour le dernier fichier)
        if i < len(files) - 1:
            print(f"Pause de {api_delay} secondes...")
            time.sleep(api_delay)
    
    # Écrire tous les résultats dans un seul fichier
    with open(os.path.join(output_dir, 'all_results.json'), 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    print(f"Traitement terminé. {success_count}/{len(files)} fichiers analysés avec succès.")
    return results

# Fonction pour comparer d'anciennes et nouvelles métadonnées (pour vérifier les améliorations)
def compare_metadata(old_dir, new_dir, focus_field="competences"):
    """Compare les métadonnées entre deux répertoires, en se concentrant sur un champ spécifique."""
    old_files = list(Path(old_dir).glob('*.json'))
    comparisons = []
    
    for old_file in old_files:
        uuid = old_file.stem
        new_file = Path(new_dir) / f"{uuid}.json"
        
        if new_file.exists():
            with open(old_file, 'r', encoding='utf-8') as f:
                old_data = json.load(f)
            
            with open(new_file, 'r', encoding='utf-8') as f:
                new_data = json.load(f)
            
            old_value = old_data[uuid].get(focus_field, [])
            new_value = new_data[uuid].get(focus_field, [])
            
            comparisons.append({
                "uuid": uuid,
                "old": old_value,
                "new": new_value
            })
    
    return comparisons

# Exemple d'utilisation (à exécuter dans une cellule séparée)
# results = process_latex_files(
#     source_dir="../src/latex/amscc",
#     output_dir="../metadata_improved",  # Utiliser un répertoire différent pour les nouvelles métadonnées
#     force_reprocess=True,               # Forcer le retraitement pour générer de nouvelles métadonnées
#     api_delay=1.5                       # Pause entre les appels API
# )

# Exemple pour comparer les anciennes et nouvelles métadonnées
# comparisons = compare_metadata("../metadata", "../metadata_improved")
# for comp in comparisons:
#     print(f"UUID: {comp['uuid']}")
#     print(f"Anciennes compétences: {comp['old']}")
#     print(f"Nouvelles compétences: {comp['new']}")
#     print("---")

In [13]:
# Exemple d'utilisation (à exécuter dans une cellule séparée)
results = process_latex_files(
    source_dir="../src/latex/amscc",
    output_dir="../metadata/amscc-v2",  # Utiliser un répertoire différent pour les nouvelles métadonnées
    force_reprocess=True,               # Forcer le retraitement pour générer de nouvelles métadonnées
    api_delay=5                       # Pause entre les appels API
)

Traitement de 460 fichiers LaTeX...

[1/460] Analyse de YBwt.tex...
Pause de 5 secondes...

[2/460] Analyse de trhY.tex...
AVERTISSEMENT: La compétence 'réaliser un test d'hypothèse statistique' ne commence pas par un verbe d'action.
Pause de 5 secondes...

[3/460] Analyse de aWAS.tex...
AVERTISSEMENT: La compétence 'tronquer un polynôme à un ordre donné' ne commence pas par un verbe d'action.
AVERTISSEMENT: La compétence 'effectuer un produit de polynômes' ne commence pas par un verbe d'action.
Pause de 5 secondes...

[4/460] Analyse de DEZs.tex...
Pause de 5 secondes...

[5/460] Analyse de 5cbY.tex...
AVERTISSEMENT: La compétence 'estimer la moyenne et la variance d'une population' ne commence pas par un verbe d'action.
Pause de 5 secondes...

[6/460] Analyse de ouYw.tex...
AVERTISSEMENT: La compétence 'manipuler des développements en série entière' ne commence pas par un verbe d'action.
AVERTISSEMENT: La compétence 'décomposer une fraction rationnelle en éléments simples' ne commenc

In [14]:
def clean_individual_json_files(metadata_dir):
    """
    Supprime tous les fichiers JSON individuels dans le répertoire donné,
    mais conserve le fichier récapitulatif 'all_results.json'.
    
    Args:
        metadata_dir (str): Chemin vers le répertoire contenant les fichiers JSON
    
    Returns:
        int: Nombre de fichiers supprimés
    """
    import os
    from pathlib import Path
    
    # Vérifier que le répertoire existe
    if not os.path.exists(metadata_dir):
        print(f"Le répertoire {metadata_dir} n'existe pas.")
        return 0
    
    # Vérifier que le fichier récapitulatif existe
    all_results_path = os.path.join(metadata_dir, 'all_results.json')
    if not os.path.exists(all_results_path):
        print(f"ATTENTION: Le fichier récapitulatif {all_results_path} n'existe pas.")
        response = input("Voulez-vous quand même continuer et supprimer les fichiers individuels? (o/n): ")
        if response.lower() != 'o':
            print("Opération annulée.")
            return 0
    
    # Compter et supprimer les fichiers individuels
    count = 0
    for json_file in Path(metadata_dir).glob('*.json'):
        # Ne pas supprimer le fichier récapitulatif
        if json_file.name != 'all_results.json':
            json_file.unlink()
            count += 1
    
    print(f"{count} fichiers JSON individuels ont été supprimés du répertoire {metadata_dir}.")
    return count

# Exemple d'utilisation:
# Pour nettoyer le répertoire contenant les métadonnées améliorées
clean_individual_json_files("../metadata/amscc-v2")

# Pour nettoyer le répertoire contenant les métadonnées originales
# clean_individual_json_files("../metadata")

459 fichiers JSON individuels ont été supprimés du répertoire ../metadata/amscc-v2.


459